# Mocking functions: Mocker

## Introducción
A veces, durante las pruebas, necesita algunos datos falsos.

Tal vez esté haciendo una llamada de API de terceros que puede ser costosa de ejecutar o tiene algún límite que no desea arriesgarse a alcanzar. O tal vez necesite crear un objeto completo sobre la marcha solo para una o dos funciones, pero ese objeto puede ser difícil de construir desde cero. O tal vez tus necesidades sean más sencillas. 

Por estas razones y más, existe la biblioteca `mock`. Actualmente estamos usando `pytest`, por lo que no tenemos que preocuparnos por acceder a mock desde la biblioteca `unittest`; podemos usar `pytest-mock`. 

Si este paquete está instalado, cuando ejecutemos `py.test` tendremos acceso al accesorio `mocker`, que ha adjuntado toda la funcionalidad de la biblioteca `unittest.mock`.

Esta no será una explicación exhaustiva de lo que se puede hacer con la biblioteca `pytest-mock`, pero cubrirá algunos de los casos de uso más comunes.

## Monkey Patching

`monkeypatch`  le permite interceptar lo que haría normalmente una función, sustituyendo su ejecución completa con un valor de retorno de su propia especificación. 

Tenga en cuenta que el `monkey patching` en una llamada de función no cuenta como una prueba de esa llamada de función. En realidad, **no está** usando la función que ha parcheado; está rechazando su comportamiento predeterminado y sustituyéndolo por un nuevo comportamiento.

Tomemos como ejemplo las siguientes funciones para obtener y analizar datos de GitHub

In [14]:
%%writefile users.py
import requests
import json

def get_user_followers(username):
    """Grab the JSON object from a given user's followers."""
    response = requests.get('https://api.github.com/users/{}/followers'.format(username))
    return response.content

def get_follower_names(username):
    """Given a username of a GitHub user, return a list of follower usernames."""
    json_out = get_user_followers(username)
    as_dict = json.loads(json_out)
    return list(map(lambda x: x["login"], as_dict))

Overwriting users.py


GitHub establece un límite en la velocidad a la que puede acceder a sus datos. Sin embargo, a medida que probamos `get_follower_names` y otro código que pueda llamar a `get_user_followers`, tendremos que llamar a esta función una y otra vez.

In [15]:
%%writefile test_users.py
from users import get_follower_names
def test_get_follower_names_returns_name_list():
    assert 'jradavenport' in get_follower_names('nhuntwalker')

Overwriting test_users.py


En poco tiempo, alcanzaremos nuestro límite de frecuencia (incluso con un token de API). Cualquier prueba que ejecutemos con esta función después de este punto fallará automáticamente.

¿Entonces lo que hay que hacer?

En nuestro archivo de prueba, podemos "parchear" la llamada a la API de GitHub. 

Podemos hacer esto usando el accesorio `monkeypatch` proporcionado por `pytest-mock`. No es necesario que lo importe al archivo. 

In [16]:
%%writefile test_users.py
import users

def substitute_func(username):
    return '[{"login": "aishapectyo"},{"login": "jradavenport"},{"login": "kridicule"}]'

def test_get_follower_names_returns_name_list(monkeypatch):
    monkeypatch.setattr(users, 'get_user_followers', substitute_func)
    assert 'jradavenport' in users.get_follower_names('nhuntwalker')

Overwriting test_users.py


`monkeypatch` es un objeto en sí mismo con una variedad de métodos para falsificar atributos de otros objetos o espacios de nombres completos. En el ejemplo anterior, usamos el método `.setattr` para intercambiar nuestra función `users.get_user_followers` real con alguna otra función sustituta, `substitute_func`. Al tratar el módulo de usuarios como un objeto, `monkeypatch` cambia el comportamiento de la función `get_user_followers` dentro del módulo cuando se llama para esta prueba.

La función sustituta, a su vez, simplemente devuelve lo que le digamos para los propósitos de la(s) prueba(s). En el ejemplo anterior, codificamos una cadena que es un objeto JSON adecuado, tal como lo espera `users.get_follower_names`. 

De lo contrario, la función sustituta no realiza ningún trabajo que no esté especificado en la definición de la función.

El resultado final es que, **para esta prueba**, siempre que hagamos la solicitud HTTP completa a GitHub para sus datos, en su lugar, obtendremos el valor de retorno de `substitute_func()`.

Fuera de esta prueba, a menos que usemos `monkeypatch` nuevamente, `users.get_user_followers` funcionará como se supone que debe hacerlo.

Como ocurre con la mayoría de los problemas de prueba, si queremos que ocurra el mismo comportamiento en una variedad de pruebas, siempre podemos configurar un `fixture`. 

Recuerde que cada vez que incluye un `fixture` en su función de prueba, el código dentro del dispositivo se ejecuta en su totalidad antes de ejecutar la prueba. Podemos usar eso a nuestro favor.

In [17]:
%%writefile test_users.py
import pytest

def substitute_func(username):
    return '[{"login": "aishapectyo"},{"login": "jradavenport"},{"login": "kridicule"}]'

@pytest.fixture
def gh_patched(monkeypatch):
    import users
    monkeypatch.setattr(users, 'get_user_followers', substitute_func)

def test_get_follower_names_returns_name_list(gh_patched):
    from users import get_follower_names
    assert 'jradavenport' in get_follower_names('nhuntwalker')

Overwriting test_users.py


Si queremos que en cada prueba este comportamiento sea parcheado sin que tengamos que pensar en ello, podemos establecer el argumento de palabra clave `autouse` de `pytest.fixture` en `True`.

In [18]:
%%writefile test_users.py
import pytest
    import users

def substitute_func(username):
    return '[{"login": "aishapectyo"},{"login": "jradavenport"},{"login": "kridicule"}]'

@pytest.fixture(autouse=True)
def gh_patched(monkeypatch):
    monkeypatch.setattr(users, 'get_user_followers', substitute_func)

def test_get_follower_names_returns_name_list():
    from users import get_follower_names
    assert 'jradavenport' in get_follower_names('nhuntwalker')

Overwriting test_users.py


Tenga en cuenta que la prueba ya no incluye nuestro `fixture` en la lista de parámetros.

Tenga cuidado con los cambios de comportamiento universalmente. Si parcheamos el comportamiento de `get_user_followers` automáticamente para cada prueba, corremos el riesgo de no poder probar la función después de todo.

## MagicMock and Faking Objects

A veces no es suficiente parchear una sola función; a veces necesitas una instancia de un objeto completo, pero construir ese objeto no es un esfuerzo trivial. 

Considere el siguiente ejemplo:

In [19]:
%%writefile request.py
def some_view(request):
    if request.method == "GET":
        return {}
    if request.method == "POST":
        new_entry = Entry(
            title = request.POST['title'],
            body = request.POST['body']
        )
        request.dbsession.add(new_entry)
        return HTTPFound(request.route_url('entry_list'))

Overwriting request.py


Aquí tenemos una función de vista que maneja las solicitudes `GET` y `POST`. Se espera como argumento algún tipo de objeto de solicitud, pero normalmente solo podemos construir objetos de solicitud a partir de una solicitud HTTP real. Por lo tanto, debemos tener configurado algún tipo de cliente de prueba que pueda enviar solicitudes o recibir solicitudes reales para probar nuestra vista.

(Ignore el hecho de que esto se basa en el patrón de `Pyramid` para las solicitudes de construcción y que `Pyramid` tiene su propio objeto `DummyRequest` integrado)

Con el objeto `MagicMock`, podemos construir un objeto que pueda actuar como una `request` sin tener que ser realmente una instancia **real** de ninguna clase de `request`. En ese objeto podemos definir cualquier método o atributo que pueda ser útil para la prueba. De esta manera, nos preocupamos menos por la configuración que implica probar una función y, en cambio, nos enfocamos solo en darle a la función lo que necesita para funcionar.

Para obtener acceso al objeto `MagicMock` de `pytest-mock`, primero tenemos que incluir el accesorio de burla que nos proporciona `pytest-mock`. El objeto `MagicMock` es un atributo de ese dispositivo y se puede utilizar como desee desde allí.

En nuestra prueba para `some_view`, podríamos escribir

In [20]:
%%writefile test_request_get.py

def test_some_view_get_req_returns_dict(mocker):
    from views import some_view
    req = mocker.MagicMock()
    req.method = 'GET'
    assert some_view(req) == {}

Writing test_request_get.py


De esta forma probamos que la función `some_view` dado un objeto con un método de `GET` devuelve el diccionario que esperamos. De manera similar, podemos escribir más pruebas asumiendo que cualquier objeto simulado por el que pasamos es el objeto real.

In [21]:
%%writefile test_request_post.py
def test_some_view_post_returns_redirect(mocker):
    from views import some_view
    req = mocker.MagicMock()
    req.method = 'POST'
    req.POST = {'title': 'some title', 'body': 'some body text'}
    req.dbsession.add = lambda arg: None
    assert isinstance(some_view(req), HTTPFound)

Overwriting test_request_post.py


Aquí hemos creado un objeto con un atributo de método que tiene un valor de 'POST', un atributo `POST` que es un diccionario que contiene algunos valores y un atributo `dbsession` que tiene su propio método falso, `add()`. 

*Ninguno de estos está obligado a funcionar realmente para que nuestra prueba pase.*

Omitimos la sobrecarga de tener que configurar esa funcionalidad solo para ejecutar estas pruebas. En cambio, nos aseguramos de que nuestro objeto falso tenga todos los atributos que necesitamos para que la función funcione. Luego lo pasamos a la función sin que la función sea más sabia.

Como podemos ver, el objeto `MagicMock` es prácticamente arcilla para esculpir, adoptando cualquier forma y funcionalidad que necesitemos por el momento. Es bueno para pruebas unitarias de funciones que no requieren que también verifiquemos los efectos secundarios. 

Dependiendo del efecto secundario que esperemos, es posible que incluso podamos burlarnos de eso proporcionando el argumento de la palabra clave `side_effect` al inicializar `MagicMock`, pero si estamos llegando al punto de probar la funcionalidad interconectada, es posible que deseemos elegir un método de prueba diferente.

## mocker.spy for Tracking Your Methods

A veces no quieres apropiarte completamente de una función. A veces, solo desea realizar un seguimiento del uso de una función en su aplicación o base de código. Para eso, hay `mocker.spy`.

`mocker.spy` permitirá que su objeto o función actúe exactamente como lo haría normalmente en todos los casos. El beneficio es que puede utilizar algunas de las características de un objeto `MagicMock` junto con las operaciones habituales de su función u objeto.

Por ejemplo, considere el siguiente objeto masivamente ineficiente

In [22]:
%%writefile visits.py

class Numbers(object):
    def __init__(self, iterable):
        self._container = iterable

    def make_unique(self):
        i = 0
        visited = []
        while i < len(self._container):
            if self._container[i] in visited:
                self._drop_val(i)
                i = 0
                continue
            visited.append(self._container[i])
            i += 1

    def _drop_val(self, idx):
        self._container.pop(idx)

Overwriting visits.py


Se debe llamar al método `_drop_val` siempre que haya un valor para eliminar de la lista de contenedores. Quizás para una lista determinada de valores (es decir, $[1,2,1,2,1,2]$) quiero asegurarme de que `_drop_val` se llame un número específico de veces (es decir, 4). Puedo incorporar eso en mis pruebas espiando `_drop_val` y verificando la cantidad de veces que se llamó a este método.

In [23]:
%%writefile test_visits.py
def test_values_are_dropped_if_already_seen(mocker):
    nums = Numbers([1,2,1,2,1,2])
    mocker.spy(nums, '_drop_val')
    nums.make_unique()
    assert nums._make_unique.call_count == 4

Overwriting test_visits.py


Y ahora, cada vez que se llamó a `_drop_val`, ese recuento se mantuvo y se puede verificar más adelante.

Además de espiar el recuento de llamadas, los métodos de espionaje tienen otras funciones útiles:

* assert_called_with(*args, **kwargs)
* assert_any_call(*args, **kwargs)
* called
* mock_calls


Consulte la [documentación de Mock Class](https://docs.python.org/3/library/unittest.mock.html) para conocer todos los detalles.

Puede usar esto de varias maneras, como encontrar cuellos de botella en la ejecución de su código buscando métodos que se llaman un montón de veces, solucionar problemas de ejecución de la función verificando con qué se llama, descubrir qué funciones / métodos están llamando al estás interesado, etc.